In [2]:
import numpy as np
import math as m
np.set_printoptions(linewidth=np.inf)
from Matr import klocal,kglobal, orgElements,kglobal2f2,solveTruss,plotResults

In [6]:
#Project Inputs
areas = np.array([[12,12,12,12,12,12,12,12,12,12,12]])
coords = np.array([[0,0],[120,0],[240,0],[360,0],[480,0],[180,-800],[300,-800]])
con = np.array([[1,2],[2,3],[3,4],[4,5],[1,6],[2,6],[3,6],[3,7],[4,7],[5,7],[6,7]])
moduli = np.array([[29,29,29,29,29,29,29,29,29,29,29]])*1000
Bc = np.array([[1,1,0],[1,2,0],[5,1,0],[5,2,0]])
P = np.array([[2,2,-20],[3,2,-35],[4,2,-15]])
stcostLF = 2
stweight = 500/(12)
n6 = 6
n7 = 7
jump1 = 0.01
jump2 = 0.005
n1 = 2
n2 = 3
n3 = 4
iterations = 10000
y6base = 173
y7base = 173
nt = 3
vals = 10000
lowval = 1
highval = 500

def randomrun(Bc,P,coords, areas, moduli,con,n6,n7,steelcostLF,steelweight,n1,n2,n3,nt,vals,lowval,highval):
    
    random_numbers4loc1 = np.random.uniform(low=lowval, high=highval, size=vals)
    random_numbers4loc2 = np.random.uniform(low=lowval, high=highval, size=vals)
    size = np.shape(random_numbers4loc2)[0]

    R_collection = np.zeros((size,3))
    R_collection_nodes =  np.zeros((size,1))
    
    for i in range(size):
        start6 = random_numbers4loc1[i]
        
        start7 = random_numbers4loc2[i]
        
        coordsM = Placeiter(coords,start6,start7)
        results = solveTruss(Bc,P,coordsM, areas, moduli, con)
        dispYall = results[7]
        lengths = results[8]
        stresses = results[9]
        dispY =  PullDisp(dispYall,n1,n2,n3,nt)
        memNum = np.shape(stresses)[0]

        num = 0
        for j in range(memNum):
            if (stresses[j] < 50):
                num = num + 1

        if (num == memNum):
            S = calcCost(dispY,lengths,steelcostLF,steelweight)
        else: 
            S = 888888888888888888888

        R_collection[i,0] = S
        R_collection[i,1] = start6
        R_collection[i,2] = start7
        R_collection_nodes[i,0] = i

    return R_collection,R_collection_nodes

def parseResults(table):
    size = np.shape(table)[0]
    baseVal = 100000000000
    index = 0
    for i in range(size):
        currentVal = table[i,0]
        if (currentVal < baseVal):
            baseVal = currentVal
            index = i
            
    configuration = table[index,:]

    return baseVal,configuration

        

def ShellRun(Bc,P,coords, areas, moduli,con,n6,n7,y6base,y7base,iterations,steelcostLF,steelweight,jump1,jump2,n1,n2,n3,nt): 
    R_collection = np.zeros((iterations,3))
    R_collection_nodes = np.zeros((iterations,1))
    for i in range(iterations):
        start6 = y6base
        start7 = y7base
        coordsM = Placeiter(coords,start6,start7)
        results = solveTruss(Bc,P,coordsM, areas, moduli, con)
        dispYall = results[7]
        lengths = results[8]
        stresses = results[9]
        dispY =  PullDisp(dispYall,n1,n2,n3,nt)
        memNum = np.shape(stresses)[0]

        num = 0
        for j in range(memNum):
            if (stresses[j] < 50):
                num = num + 1

        if (num == memNum):
            S = calcCost(dispY,lengths,steelcostLF,steelweight)
        else: 
            S = 888888888888888888888

        R_collection[i,0] = S
        R_collection[i,1] = start6
        R_collection[i,2] = start7
        R_collection_nodes[i,0] = i
        
        y6base = y6base + jump1
        y7base = y7base + jump2

    return R_collection,R_collection_nodes

def PullDisp(dispYall,n1,n2,n3,nt):
    cont = np.zeros((nt,1))
    cont[0,0] = dispYall[n1-1,0]
    cont[1,0] = dispYall[n2-1,0]
    cont[2,0] = dispYall[n3-1,0]
    return cont


def Placeiter(coords,y6,y7): 
    coords[n6-1,1] = y6
    coords[n7-1,1] = y7
    return coords

def calcCost(deflections,lengths,steelcostLF,steelweight):
    totalLengths = np.sum(lengths)
    totalWeight = steelweight * totalLengths
    C = steelcostLF * totalWeight
    sumDeflections = np.sum(deflections)
    S = np.absolute(sumDeflections) * C
    return S

table1 = ShellRun(Bc,P,coords, areas, moduli,con,n6,n7,y6base,y7base,iterations,stcostLF,stweight,jump1,jump2,n1,n2,n3,nt)[0]
table2 = randomrun(Bc,P,coords, areas, moduli,con,n6,n7,stcostLF,stweight,n1,n2,n3,nt,vals,lowval,highval)[0]

bestConfig1 = parseResults(table1)[0]
bestConfig1a = parseResults(table1)[1]

bestConfig2 = parseResults(table2)[0]
bestConfig2a = parseResults(table2)[1]

print(bestConfig1,bestConfig1a)
print(bestConfig2,bestConfig2a)



31436.47474921718 [31436.47474922   175.01         174.005     ]
31453.192948517884 [31453.19294852   172.19902584   169.65506469]
